In [0]:
df = spark.table("workspace.default.imdb_bronze")
df.show(3)

+----------+-------------+---------+---------+-----------------+--------------+-------+
|    nconst|  primaryName|birthYear|deathYear|primaryProfession|knownForTitles|country|
+----------+-------------+---------+---------+-----------------+--------------+-------+
|nm10687012|   Greg Busch|       \N|       \N|               \N|     tt4305162|  India|
|nm10687013|    Cris Kohl|       \N|       \N|               \N|     tt4305162|  India|
|nm10687014|Dave Phillips|       \N|       \N|               \N|     tt4305162|  India|
+----------+-------------+---------+---------+-----------------+--------------+-------+
only showing top 3 rows


In [0]:
df2 = df.withColumn(
    "is_actor",
    df.primaryProfession.contains("actor") | df.primaryProfession.contains("actress")
)

In [0]:
df_model = df2.select("birthYear", "deathYear", "is_actor").dropna()

In [0]:
df_pd = df_model.toPandas()
df_pd["is_actor"] = df_pd["is_actor"].astype(int)

In [0]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Train/test split
X = df_pd[["birthYear", "deathYear"]]
y = df_pd["is_actor"]
X_train, X_test, y_train, y_test = train_test_split(X, y)

with mlflow.start_run():
    # Params
    n_estimators = 50
    max_depth = 5

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

    # Train model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    model.fit(X_train, y_train)

    # Accuracy
    acc = accuracy_score(y_test, model.predict(X_test))
    mlflow.log_metric("accuracy", acc)

    # Log model
    mlflow.sklearn.log_model(model, "rf_imdb_actor_model")

2025-06-24 19:09:26,737 2857 ERROR _handle_rpc_error GRPC Error received
Traceback (most recent call last):
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py", line 1853, in config
    resp = self._stub.Config(req, metadata=self.metadata())
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/grpc/_interceptor.py", line 277, in __call__
    response, ignored_call = self._with_call(
                             ^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/grpc/_interceptor.py", line 332, in _with_call
    return call.result(), call
           ^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/grpc/_channel.py", line 440, in result
    raise self
  File "/databricks/python/lib/python3.11/site-packages/grpc/_interceptor.py", line 315, in continuation
    response, call = self._thunk(new_method).with_call(
                     ^^^^^^^^^^^^^^

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5369713671543925>, line 12
      9 y = df_pd["is_actor"]
     10 X_train, X_test, y_train, y_test = train_test_split(X, y)
---> 12 with mlflow.start_run():
     13     # Params
     14     n_estimators = 50
     15     max_depth = 5

File /databricks/python/lib/python3.11/site-packages/mlflow/tracking/fluent.py:312, in start_run(run_id, experiment_id, run_name, nested, tags, description, log_system_metrics)
    304 if len(_active_run_stack) > 0 and not nested:
    305     raise Exception(
    306         (
    307             "Run with UUID {} is already active. To start a new run, first end the "
   (...)
    310         ).format(_active_run_stack[0].info.run_id)
    311     )
--> 312 client = MlflowClient()
    313 if run_id:
    314     existing_run_id = run_id

File /databricks/python/lib/python3.11/site-packages/mlflo